In [ ]:



# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn 
import keras
import tensorflow as tf
import seaborn as sns
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


## data preparation

### load the data

In [ ]:

# get the data from its path 
data =  pd.read_csv('../input/cardiovascular-disease-dataset/cardio_train.csv',sep=';', index_col = "id")
data['age']= data['age']/365
data['age']= data['age'].astype(int)
#name_group = ['<40', '40: 55' ,'>55']
#data['gender'] = data['gender'].astype(str)
#data['gluc'] = data['gluc'].astype(str)
#data['cholesterol'] = data['cholesterol'].astype(str)
#data['age_class'] = pd.qcut(data['age'],q=3 ,labels = name_group)
#data = pd.get_dummies(data )#,drop_first = True)
data['BMI'] = data['weight']/(np.power(data['height']/100, 2))
data['BMI'] = data['BMI'].astype(int)
data['Cardio_disease'] = data['cardio']
data = data.drop(columns = ['height','cardio','weight'])
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()# get the statistical information of the data

In [ ]:
# get the correlation of tbetween cardiovascular disease and each risk factor
plt.figure(figsize=(16, 8))
sns.heatmap(data.corr(), annot=True, fmt='.3f')



### detect outliers

In [ ]:

#detect the outliers through box plots 

df_melt = pd.melt(frame= data, id_vars='Cardio_disease',
                  value_vars=[ 'ap_hi' , 'ap_lo' , 'BMI','age'],
                  var_name='features',
                  value_name='value', 
                  col_level=None)
# Box plot
plt.figure(figsize=(12, 10))
sns.boxplot(x='features', y='value', hue='Cardio_disease', data=df_melt)
plt.xticks(rotation=90)

### get the inter-quartile range 

In [ ]:
# Diciding upper and lower bound 
ap_list = ['ap_hi', 'ap_lo','BMI', ]
df_ap = pd.DataFrame(index=['lower_bound', 'upper_bound'])

for i in ap_list:
    Q1 = data[i].quantile(0.25)
    Q3 = data[i].quantile(0.75)
    IQR = Q3 - Q1
    
    lowr_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    df_ap[i] = [lowr_bound, upper_bound]
    
df_ap.head()


###  remove the outliers

In [ ]:
# remove the outliers 
outlier_filter = ((data["ap_hi"]>170) |(data["ap_hi"]<90)| (data["ap_lo"]>105)|(data["ap_lo"]< 65) |(data["BMI"]> 40)|(data["BMI"]< 12) )
data = data[~outlier_filter]
data.shape

In [ ]:
# box plot after deleting the outliers 
df_melt = pd.melt(frame= data, id_vars='Cardio_disease',
                  value_vars=['ap_hi' , 'ap_lo' , 'BMI'],
                  var_name='features',
                  value_name='value', 
                  col_level=None)
# Box plot
plt.figure(figsize=(10 ,13))
sns.boxplot(x='features', y='value', hue='Cardio_disease', data=df_melt)
plt.xticks(rotation=90)

### data standarization


In [ ]:
# data standarization
std_list = ["age" ,"BMI","ap_hi", "ap_lo"] 

def standartization(x):
    x_std = x.copy(deep=True)
    for column in std_list:
        x_std[column] = (x_std[column] - x_std[column].mean()) / x_std[column].std() 
    return x_std

data= standartization(data)
data.head()


In [ ]:
# get the correlation of tbetween cardiovascular disease and each risk factor after preprocessing
plt.figure(figsize=(16, 8))
sns.heatmap(data.corr(), annot=True, fmt='.3f')

### specify the features 

In [ ]:
X = data.iloc[:, :-1]
X.head()

In [ ]:
y = data.iloc[: ,-1]
y.head()

### splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test ,y_train, y_test = train_test_split(X,y,
                                                   test_size=0.1,
                                                   random_state=0,
                                                  shuffle = True,
                                                  stratify = y)

print('training data shape is :{}.'.format(X_train.shape))
print('training label shape is :{}.'.format(y_train.shape))
print('testing data shape is :{}.'.format(X_test.shape))
print('testing label shape is :{}.'.format(y_test.shape))


### reduce the dimensions

In [ ]:
# dimensionality reduction to consider the most effective factors to the model 
from sklearn.decomposition  import PCA
pca =PCA(n_components = 5)# the same number you define as input_dim argument of first layer of ann
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
explained_variance= pca.explained_variance_ratio_
print(explained_variance)


### 1 - Artificial Neural Networks (ANN ) model

In [ ]:

# ANN model 

model = Sequential() 

model.add(Dense(32,  activation = 'relu' ,input_dim = 5))
model.add(Dense(16,  activation = 'relu' ))
model.add(Dense(7 , activation = 'relu' ))

model.add(Dense(1  ,activation = 'sigmoid'))

model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' ,metrics = ['accuracy'])

num_epochs = 20
history = model.fit(X_train ,
                    y_train ,
                    epochs= num_epochs ,
                    batch_size = 50,
                    validation_data=(X_train ,y_train))

# evaluate the model
loss, acc = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy: {:3.3f} and loss {:3.3f}'.format(acc ,loss))

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()
    
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) *1
n = 10
right = 0
wrong = 0
prediction = y_pred.tolist()
prediction_sample = prediction[:n]
y_true = [[i] for i in y_test]
true_sample = y_true[:n]
print(f'\n accuracy : {acc}\n')
print(f' Actual values   : {true_sample}')
print(f'predicted values : {prediction_sample}\n')

for i in range(n):

    if true_sample[i] == prediction_sample[i]:
        right = right +1
    else :
        wrong= wrong + 1
        
print(f'{right} of {n} True Prediction')
print(f'{wrong} of {n} False Prediction')

In [ ]:
from sklearn.metrics import confusion_matrix ,ConfusionMatrixDisplay
cm = confusion_matrix(y_true ,prediction)
disp = ConfusionMatrixDisplay(cm , display_labels=None)
disp.plot()

In [ ]:
#save the model 

"""
tf.keras.models.save_model(
    model, filepath, overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
"""

### 2- Ensemble Method (Rrandom Forest )


In [ ]:
from sklearn.ensemble import  RandomForestClassifier
from sklearn.model_selection import  GridSearchCV


random_forest = GridSearchCV(estimator=RandomForestClassifier(),
                             param_grid={'n_estimators': [100, 300]}, cv=5).fit(X_train, y_train)
# fit the training data
random_forest.fit(X_train, y_train)

acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)

print(acc_random_forest,random_forest.best_params_)

acc_test_random_forest = round(random_forest.score(X_test, y_test) * 100, 2)


# scores
print(f"Training Score: {acc_random_forest}")
print(f"Testing Score: {acc_test_random_forest}")



### 3- logistic regression with semi_supervisor learning with pytorch 

Semi-supervised learning is the type of machine learning that uses a combination of a small amount of labeled data and a large amount of unlabeled data to train models. This approach to machine learning is a combination of supervised machine learning, which uses labeled training data, and unsupervised learning, which uses unlabeled training data


-  Unlabeled data, when used in conjunction with a small amount of labeled data, can produce considerable      improvement in learning accuracy


In [ ]:
import numpy as np
import math
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression

from sklearn.utils import shuffle
from sklearn.metrics import f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F 
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as dsets

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [ ]:
# get the data from its path 
df=pd.read_csv("E:\cadio\cardio_train\cardio_train.csv",sep=";", index_col = 'id')
df = shuffle(df)
df.head()

In [ ]:
# standarizeation of data
columns = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
mean_std = {}

for column in columns:
    mean_std[column] = (df[column].mean(), df[column].std())
    df[column] = (df[column] - df[column].mean())/df[column].std()

In [ ]:
class CardioDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        x = self.features[idx]
        y = self.labels[idx]
        
        x = torch.tensor(x, dtype=torch.float)
        y = torch.tensor(np.eye(2)[y], dtype=torch.long)
        return x, y
    
    
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim=11, output_dim=2):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [ ]:
# Generate indices for splits
test_ind = round(len(df)*0.25)
train_ind = test_ind + round(len(df)*0.01)
unlabeled_ind = train_ind + round(len(df)*0.74)


# Partition the data
test = df.iloc[:test_ind]
train = df.iloc[test_ind:train_ind]
unlabeled = df.iloc[train_ind:unlabeled_ind]
print('testing data shape is :{}.'.format(test.shape))
print('training data shape is :{}.'.format(train.shape))
print(' unlabel shape is :{}.'.format(unlabeled.shape))


In [ ]:
def test_train_unlabeled(test, train, unlabeled):
    X_train_ = train.drop('cardio', axis=1)
    X_train_ = torch.tensor(X_train_.values)

    y_train_ = train.cardio
    y_train_ = torch.tensor(y_train_.values)

    X_unlabeled = unlabeled.drop('cardio', axis=1)
    X_unlabeled = torch.tensor(X_unlabeled.values)
    X_unlabeled = X_unlabeled.type(torch.float)

    X_test_ = test.drop('cardio', axis=1)
    X_test_ = torch.tensor(X_test_.values)

    y_test_ = test.cardio
    y_test_= torch.tensor(y_test_.values)
    
    return X_train_, y_train_, X_unlabeled, X_test_, y_test_

X_train_, y_train_, X_unlabeled, X_test_, y_test_ = test_train_unlabeled(test, train, unlabeled)

In [ ]:
model = LogisticRegression()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
nr_episodes = 8

for ep in range(nr_episodes):

    X_train_, y_train_ = shuffle(X_train_, y_train_)
    X_test_, y_test_ = shuffle(X_test_, y_test_)

    train_ds = CardioDataset(X_train_.clone().detach(), y_train_.clone().detach())
    test_ds  = CardioDataset(X_test_.clone().detach(),  y_test_.clone().detach())

    train_loader = DataLoader(train_ds, batch_size=8)
    test_loader = DataLoader(train_ds, batch_size=8)

    nr_epochs = 100

    for e in range(nr_epochs):

        epoch_accs = 0
        epoch_loss = 0

        val_epoch_accs = 0
        val_epoch_loss = 0

        for i, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            x = x.type(torch.FloatTensor)
            y = y.type(torch.FloatTensor)
            y_max = y.max(axis=1)[1]
            
            #Train Logistic Regression classifer on the labeled trainning data.
            outputs = model(x)
            acc = sum(outputs.max(axis=1)[1] == y_max).item()
            acc /= len(x)
            epoch_accs += acc / len(train_loader)

            loss = criterion(outputs, y_max)
            epoch_loss += loss / len(train_loader)

            loss.backward()
            optimizer.step()

        for i, (x, y) in enumerate(test_loader):
            x = x.type(torch.FloatTensor)
            y = y.type(torch.FloatTensor)
            y_max = y.max(axis=1)[1]

            outputs = model(x)
            acc = sum(outputs.max(axis=1)[1] == y_max).item()
            acc /= len(x)
            val_epoch_accs += acc / len(test_loader)

            loss = criterion(outputs, y_max)
            val_epoch_loss += loss / len(test_loader)

        if((e+1) % 40 == 0):
            print('i:{:3d}, Epoch:{:4d}, train_loss:{:1.3f}, ' \
                  'epoch_acc:{:1.3f}, val_loss:{:1.3f}, val_accs:{:1.3f}'
                      .format(ep+1, e+1, epoch_loss, epoch_accs,
                              val_epoch_loss, val_epoch_accs))
            
         
    # add supervised features, labels
    logits = model(X_unlabeled)
    pred_probs = F.softmax(logits, dim=1)
    preds = torch.argmax(logits, dim=1)
    
    # Separate predictions with > 99% probability
    high_prob_indices = torch.where(pred_probs > (0.9 + ep * 0.01))
    
    #Concatenante the pseudo-labeled data with the labeled training data.
    
    X_train_ = torch.cat((X_train_, X_unlabeled[high_prob_indices[0]]), 0)
    y_train_ = torch.cat(([y_train_, high_prob_indices[1]]), 0)
    
    #Use trained classifer to make predictions for the labeled test data.
    
    new_indexes = [a for a in np.arange(len(X_unlabeled)) 
                       if a not in high_prob_indices[0] ]
    X_unlabeled = X_unlabeled[new_indexes]
    
    print('------------- Adding {:}# of features, ' \
          '{:}# of unlabeled remaining -------------'
            .format(len(high_prob_indices[0]), len(X_unlabeled)))